# Welcome to the Gradient Descent workbook.

## Live Gradient Descent project

This notebook is designed to **Create a functioning workflow for numerical optimasation of controll paramiters of the **FENN** code. 

### Getting Started

### Contact

For any questions please reach out to **Adam Cole** (acole32@vols.utk.edu)


In [124]:
import numpy as np
import matplotlib.pyplot as plt
import os
import re

#Establish domain

#Generate random point

#Create "stepped" points 

#Shell commands:
#Modifly FENN
#Compile FENN

#Extract data, delete the rest

#Evaluate cost fuction

#Perform gradient descent/ find updated parameters

#Repeat from step 3

#Check for convergence

#Plots

file_path = "/home/ojc/Downloads/UTKCAP/FENN/simulation_log.txt"

In [125]:
#Establish domain

EpsA_range=(1.0e-02, 1.0e-06)#Hi to Low
EpsR_range=(1.0e-02, 1.0e-06) 

EpsA_range = [float(x) for x in EpsA_range]
EpsR_range = [float(x) for x in EpsR_range]

print(EpsA_range)
print(EpsR_range)

#pick random points
EpsA= np.random.uniform(*EpsA_range)
EpsR= np.random.uniform(*EpsR_range)

print(EpsA)
print(EpsR)

#tune accuracy
n = 2

Step = (max(EpsA_range)-min(EpsA_range))/n

#Other parameters

'''learning_rate = 0.9
Convergence_Tol = 0.9'''
i = 0.0

[0.01, 1e-06]
[0.01, 1e-06]
0.009924081728750922
0.002428153661318444


In [126]:
#Change working directory!

os.chdir('/home/ojc/Downloads/UTKCAP/FENN')

#Convergence Test Loop

while True:

#Initialize

#Step points

    EpsA_forward = EpsA + Step
    EpsR_forward = EpsR + Step

    EpsA_backward = EpsA - Step
    EpsR_backward = EpsR - Step
    
#Shell Scripts- for BOTH step forward and back


#---------------------------------------------- Forward ------------------------------------------------------

# Modify Networks.cpp to update EpsA and EpsR values

    !sed -i "s/double EpsA = [0-9.e+-]*;/double EpsA = {EpsA_forward};/" /home/ojc/Downloads/UTKCAP/FENN/Networks.cpp
    !sed -i "s/double EpsR = [0-9.e+-]*;/double EpsR = {EpsR_forward};/" /home/ojc/Downloads/UTKCAP/FENN/Networks.cpp

#Compile FENN
    
    !g++ /home/ojc/Downloads/UTKCAP/FENN/Networks.cpp -o FENN \
        -I/home/ojc/anaconda3/pkgs/hdf5-1.10.6-h3ffc7dd_1/include \
        -I/usr/include/eigen3 \
        -L/home/ojc/anaconda3/pkgs/hdf5-1.10.6-h3ffc7dd_1/lib \
        -Wl,-rpath,/home/ojc/anaconda3/pkgs/hdf5-1.10.6-h3ffc7dd_1/lib \
        -lhdf5_cpp -lhdf5
    
    !rm -f /home/ojc/Downloads/UTKCAP/FENN/simulation_log.txt
    
    !./FENN > /dev/null 2>&1
    
#Read from FENN/ Extract t and ERR - Based off of Perform_Experiment.sh
    
    with open(file_path, 'r') as file:
        for line in file:
            if "Wall-clock time:" in line:
                t_forward = line.split(":")[1].strip() 
            if "Accumulated Error:" in line:
                ERR_forward = line.split(":")[1].strip() 
                
#Fixing time units
                
#Forward
    
    print(t_forward)
    
    if isinstance(t_forward, float):
        continue  
    
    value, unit = t_forward.split()
    
    if unit == 'microseconds':
            unit = 1.0e-6
    else:
        print('error in time units')
    
    value = float(value)
    t_forward = value * unit

    print(t_forward)

    ERR_forward = float(ERR_forward)

#---------------------------------------------- Backward ------------------------------------------------------
                
# Modify Networks.cpp to update EpsA and EpsR values

    !sed -i "s/double EpsA = [0-9.e+-]*;/double EpsA = {EpsA_backward};/" /home/ojc/Downloads/UTKCAP/FENN/Networks.cpp
    !sed -i "s/double EpsR = [0-9.e+-]*;/double EpsR = {EpsR_backward};/" /home/ojc/Downloads/UTKCAP/FENN/Networks.cpp

#Compile FENN
    
    !g++ /home/ojc/Downloads/UTKCAP/FENN/Networks.cpp -o FENN \
        -I/home/ojc/anaconda3/pkgs/hdf5-1.10.6-h3ffc7dd_1/include \
        -I/usr/include/eigen3 \
        -L/home/ojc/anaconda3/pkgs/hdf5-1.10.6-h3ffc7dd_1/lib \
        -Wl,-rpath,/home/ojc/anaconda3/pkgs/hdf5-1.10.6-h3ffc7dd_1/lib \
        -lhdf5_cpp -lhdf5
    
    !rm -f /home/ojc/Downloads/UTKCAP/FENN/simulation_log.txt
    
    !./FENN > /dev/null 2>&1
    
#Read from FENN/ Extract t and ERR - Based off of Perform_Experiment.sh
    
    with open(file_path, 'r') as file:
        for line in file:
            if "Wall-clock time:" in line:
                t_backward = line.split(":")[1].strip() 
            if "Accumulated Error:" in line:
                ERR_backward = line.split(":")[1].strip()
                
#Fixing time units
    
#Backward
    
    print(t_backward)
    
    if isinstance(t_backward, float):
        continue  
    
    value, unit = t_backward.split()
    
    if unit == 'microseconds':
            unit = 1.0e-6
    else:
        print('error in time units')
    
    value = float(value)
    t_backward = value * unit

    print(t_backward)

    ERR_backward = float(ERR_backward)

#Compute the Cost function  

#Forward
    speed_forward = (1-(1/t_forward))
    
    Cost_forward = np.sqrt(speed_forward**2 + ERR_forward**2)

#Backward
    
    speed_backward = (1-(1/t_backward))
    
    Cost_backward = np.sqrt(speed_backward**2 + ERR_backward**2)

#Centered Difference Approximation of f'

    Cost_grad = (Cost_forward - Cost_backward)/(2*Step)

#Evaluate Gradient Descent

    EpsA = EpsA - learning_rate * Cost_grad
    EpsR = EpsR - learning_rate * Cost_grad

#Step
    
    i = i + 1
    
#Convergence Test

    if abs(Cost_grad) < Convergence_Tol:
        break #CONVERGENCE TEST
        

453494 microseconds
0.45349399999999995
1539491 microseconds
1.539491
571288 microseconds
0.571288
983237 microseconds
0.9832369999999999


In [127]:
#3D plots

x = EpsA_points
y = EpsR_points
z = Cost_array

# Create a 3D figure
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Create a 3D scatter plot
ax.plot(x, y, z)

# Set labels
ax.set_xlabel('EpsA')
ax.set_ylabel('EpsR')
ax.set_zlabel('Cost')

# Show the plot
plt.show()

NameError: name 'EpsA_points' is not defined